# Step 8: V05 — Transit shape (U vs V)

Goal: assess whether the transit shape looks more U-like (planet-like) or V-like (often associated with grazing eclipsing binaries).

This check fits a simple trapezoid-like model and reports a shape metric (e.g., flat-bottom duration fraction).


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False)
r = session.run('V05')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': r.flags,
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "depth_bottom": 0.000299,
    "depth_edge": 0.000235,
    "depth_ppm": 256.5,
    "method": "trapezoid_grid_search",
    "n_baseline": 35680,
    "n_bottom_points": 243,
    "n_edge_points": 488,
    "n_in_transit": 1165,
    "shape_metric_uncertainty": 0.0,
    "shape_ratio": 1.272,
    "status": "ok",
    "t_flat_hours": 4.046,
    "t_total_hours": 4.046,
    "tflat_ttotal_ratio": 1.0,
    "tflat_ttotal_ratio_err": 0.0,
    "transit_coverage": 1.0
  },
  "status": "ok"
}
```

</details>


In [ ]:
# Plot V05: Transit shape (U vs V)
out = {
    'status': r.status,
    'flags': r.flags,
}

try:
    import matplotlib.pyplot as plt
    from tess_vetter.api import plot_v_shape
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE and (r.status == 'ok') and getattr(r, 'raw', None) and (r.raw or {}).get('plot_data'):
    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='08_v05_v_shape')
    run_path = run_out_dir / 'V05_v_shape.png'
    docs_path = (docs_out_dir / 'V05_v_shape.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(8, 5))
    plot_v_shape(r, ax=ax)
    ax.set_title('V05: Transit shape (U vs V)')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/08_v05_v_shape/V05_v_shape.png`

![V05: Transit shape (U vs V)](../artifacts/tutorial_toi-5807-incremental/08_v05_v_shape/V05_v_shape.png)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/08_v05_v_shape/V05_v_shape.png",
  "flags": [],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/08_v05_v_shape/V05_v_shape.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** trapezoid fit returns tflat/ttotal ≈ **1.00** and shape_ratio ≈ **1.27** (binned profile looks flat-bottomed).
- **Why it’s useful:** grazing EBs tend to be strongly V-shaped; we don’t see that pattern here.
- **Interpretation:** consistent with a planet-like transit shape at this SNR.
- **Next step:** move to catalog-context checks (V06–V07).

</details>
